In [ ]:
#Install libraries
!pip install transformers datasets evaluate sacrebleu transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
#Import libraries
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, pipeline, EarlyStoppingCallback
from evaluate import evaluator
import evaluate
import numpy as np
import torch
from datasets import DatasetDict
import sentencepiece

In [ ]:
#load the dataset for finetuning
bsd_data = load_dataset("bsd_ja_en")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2120 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2051 [00:00<?, ? examples/s]

Dataset bsd_ja_en downloaded and prepared to /root/.cache/huggingface/datasets/bsd_ja_en/default/1.0.0/584d1d2fee129f4799fccf10cb83c7cc0a7e6147094435909c3932c4380ddb24. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
#Load the pretrained tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-jap")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-jap")
device = "cuda:0" if torch.cuda.is_available() else "cpu"
#mount the pretrained model on the GPU
model.to(device)

/usr/local/lib/python3.9/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(46276, 512, padding_idx=46275)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(46276, 512, padding_idx=46275)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0): MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLUActivation()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1

In [ ]:
#Function to see how the model performs on translating some simple sentences, it will output the original English
#sentence the model's translation 
def test_simple_text(model,tokenizer):
  #The simple sentences
  simple_example_text = ["My cat is red.","I like apples.","What is your name?","I go running everyday.","You should study more."]
  #Feed to tokenizer to get the input ids
  inputs_ids = tokenizer(simple_example_text, return_tensors="pt", padding=True).to(device)
  #Get model outputs
  outputs = model.generate(**inputs_ids)
  i = 0
  #Look over the sentences and print out the original + the translation
  for output in outputs:
    print("Source example sentence: ", simple_example_text[i])
    print("Translation of example sentence: ", tokenizer.decode(outputs[i], skip_special_tokens=True))
    i += 1

test_simple_text(model,tokenizer)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Source example sentence:  My cat is red.
Translation of example sentence:  わたし の かおり は 赤 く,
Source example sentence:  I like apples.
Translation of example sentence:  わたし は ともしび 皿 の よう に 彫り の よう に 言 っ て い ま す.
Source example sentence:  What is your name?
Translation of example sentence:  あなた の 名 は なん と い う の か.
Source example sentence:  I go running everyday.
Translation of example sentence:  わたし は すべて の 日 に 走 っ て 行 き,
Source example sentence:  You should study more.
Translation of example sentence:  あなた がた は, なお 多く の 事 を 示 す よう に と 決定 し た.


These are NOT good translations despite being very simple sentences, better translations for the sentences would be as follows (in the same order as above):
*   私の猫は赤いです。
*   私は林檎が好きです。or  林檎が好きです。
*   お名前はなんですか。
*   私は毎日ラーニングをします。or  毎日ラーニングをします。
*   あなたはもっと勉強するべきです。or  もっと勉強するべきです。


In [ ]:
src_tag = 'en_sentence'
tgt_tag = 'ja_sentence'
#Define preprocessing function to prepare and tokeninze the dataset
def preprocess(data):
  model_inputs = tokenizer(data[src_tag], text_target=data[tgt_tag], padding=True)
  return model_inputs

bsd_data_tokenized = bsd_data.map(preprocess, batched=True)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2120 [00:00<?, ? examples/s]

Map:   0%|          | 0/2051 [00:00<?, ? examples/s]

In [ ]:
#Use bleu score as an evaluation metric
bleu = evaluate.load("sacrebleu", tokenizer = 'ja-mecab')

#Function to clean the decoded predictions and labels
def clean_texts(preds, labels):
  preds = [pred.strip() for pred in preds]
  labels = [[label.strip()] for label in labels]
  return preds, labels

#Function to compute the bleu score during training
def compute_metrics(pred_labels):
  preds, labels = pred_labels
  if isinstance(preds, tuple):
      preds = preds[0]
  decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  decoded_preds, decoded_labels = clean_texts(decoded_preds, decoded_labels)

  result = bleu.compute(predictions=decoded_preds, references=decoded_labels)
  result = {"bleu": result["score"]}
  return result

In [ ]:
#Define the data collator to be used by the trainer object
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
#Some simple training arguments for finetuning
training_args = Seq2SeqTrainingArguments(
    output_dir="bsd_opus_mt",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=20,
    predict_with_generate=True,
    fp16=True,
    metric_for_best_model="bleu",
    load_best_model_at_end=True,
    greater_is_better = True,
    save_strategy = "epoch"
)

#Define the trainer object
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=bsd_data_tokenized["train"],
    eval_dataset=bsd_data_tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 2)]
)


In [ ]:
#Check the bleu score on the test set to have a comparison for after finetuning
prefintuned_results = trainer.evaluate(eval_dataset=bsd_data_tokenized["test"])

In [ ]:
print("Test set bleu score before finetuning: ", prefintuned_results["eval_bleu"])

Test set bleu score before finetuning:  0.0013795709441450164


This is a terrible BLEU score for a pretrained MT model, this dataset isn't general, but the model should still do much better than this

In [ ]:
#Finetune the model
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Bleu
1,1.266300,1.029620,1.267218
2,1.042800,0.938159,0.984572
3,0.971700,0.897003,2.630857
4,0.883500,0.867913,7.367027
5,0.829200,0.855751,0.000000
6,0.782600,0.844263,7.351506


TrainOutput(global_step=7500, training_loss=0.9927501180013021, metrics={'train_runtime': 1511.3284, 'train_samples_per_second': 264.668, 'train_steps_per_second': 16.542, 'total_flos': 2513334945447936.0, 'train_loss': 0.9927501180013021, 'epoch': 6.0})

In [ ]:
#Check the bleu score on the test set to see if it has improved
postfintuned_results = trainer.evaluate(eval_dataset=bsd_data_tokenized["test"])

In [ ]:
print("Test set bleu score after finetuning: ", postfintuned_results["eval_bleu"])

Test set bleu score after finetuning:  7.384424896029306


The BLEU is much better now after training as expected, but still one would expect it to be higher if the original model was better

In [ ]:
#Retest those simple sentences to see if there was an improvment
test_simple_text(model,tokenizer)

Source example sentence:  My cat is red.
Translation of example sentence:  私のデザインはすごくす
Source example sentence:  I like apples.
Translation of example sentence:  うんちょっと
Source example sentence:  What is your name?
Translation of example sentence:  名前は何ですか?
Source example sentence:  I go running everyday.
Translation of example sentence:  毎日遅れます
Source example sentence:  You should study more.
Translation of example sentence:  もっと学ぶべきですね


/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


These translations are a bit better, the third and fifth sentences are much better than before and are pretty close to what they should be. The second and fourth sentences are about as bad before, and the first sentences is actually a little worse than the original pretrained model translation.